In [12]:
from __future__ import print_function
import pandas as pd
from config import *
from functions import *
import datetime
import os
import sys
import getopt
import pickle
import os.path
from env import *
from termcolor import colored, cprint
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

def prepp_df_dict(_dict, org_header):
    checks_out = True
    res = {}
    header = []
    for key, value in _dict.items():
        res[value] = []
        header.append(key)
        if not key in org_header:
            print(f'{key} not checking out')
            checks_out = False
    return res, header, checks_out

def get_sheet_values_service(service, ID, _range):
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=ID, range=_range).execute()
    return result.get('values', [])

def get_sheet_as_df(service, ID, _range, col_map):
    """
    Hämtar ett sheet från Drive med id ID inom range _range och med
    header mapping enlig col_map.
    
    Returnerar DataFrame
    """
    
    values = get_sheet_values_service(service, ID, _range)
    # Den faktiska headern i sheeten
    org_header = values[0]
    
    if not values:
        print('No data found in ' + ID + ', range ' + _range + ' with col_map ' + col_map)
    else:
        errors = []
        
        # Preparerar data:
        # _dict (dict) får keys utefter col_map med tom lista som value för varje key.
        # header (list) sätts utefter col_map (vilka rubriker som skall extraheras)
        # checks_out (boolean) True om col_map keys finns med i org_header annars False
        _dict, header, checks_out = prepp_df_dict(col_map, org_header)
        
        if checks_out:
            for i, row in enumerate(values):
                if i == 0:
                    # Skippar rubrikraden
                    pass
                else:
                    try:
                        for key in header:
                            # print(f'len(row): {len(row)}')
                            # print(f'org_header.index(key): {org_header.index(key)}')
                            if len(row) <= org_header.index(key):
                                """
                                Raden som plocks ut (row) blir inte alltid lika lång som rubrikraden utan klipps av till kortare längd om celler mot
                                slutet av raden är tomma. Detta gör att man måste kolla om raden är kortare än headerraden för denna rubriken (key)
                                och om så är fallet appenda tom cell (''). Detta för att undvika index out of range error.
                                """
                                _dict[col_map[key]].append('')
                            else:
                                _dict[col_map[key]].append(row[org_header.index(key)])
    
                    except Exception as e:
                        errors.append(row)
                            
        else:
            print(f'{col_map} and {org_header} doesn\'t match')
            exit()
    
    return pd.DataFrame.from_dict(_dict), errors

def find_language(pn, name, df_sva_sv):

    language = ""
    message = "OK"
    try:
        language_series = df_sva_sv[df_sva_sv['Personnummer'].str.contains(pn)]
        if len(language_series.index) == 1:
            language = str(language_series['Språk'].values[0])
        else:
            language_series = df_sva_sv[df_sva_sv['Namn'].str.contains(name)]
            if len(language_series.index) == 1:
                email = str(language_series['Språk'].values[0])
            else:
                message = "KONTOT SAKNAS I SVA_SV SHEET"
    except Exception as e:
        print("Error while looking for email in edukonto sheet ", e)

    return language, message

def check_language(service, df_elevlista, df_sva_sv, ELEVLISTA_ID):
    elevlista_klass_list = df_elevlista.loc[:, "Klass"].tolist()
    elevlista_namn_list = df_elevlista.loc[:, "Namn"].tolist()
    elevlista_groups_list = df_elevlista.loc[:, "Grupper"].tolist()
    elevlista_personnummer_list = df_elevlista.loc[:, "Personnummer"].tolist()
    elevlista_email_list = df_elevlista.loc[:, "Mail"].tolist()

    sva_sv_namn_list = df_sva_sv.loc[:, "Namn"].tolist()
    sva_sv_personnummer_list = df_sva_sv.loc[:, "Personnummer"].tolist()
    sva_sv_language_list = df_sva_sv.loc[:, "Språk"].tolist()

    missing_language_klass = [] # for error handling. Will build df and then save missing mails to csv
    missing_language_name = []
    content = []
    for i, current_pn in enumerate(tqdm(elevlista_personnummer_list, desc="Checking language")):
        row = []
        current_name = elevlista_namn_list[i]
        current_klass = elevlista_klass_list[i]
        current_groups = elevlista_groups_list[i]
        current_email = elevlista_email_list[i]
        language, message = find_language(current_pn, current_name, df_sva_sv)
        if language == "":
            pass
        if message != "OK":
            if current_klass in relevent_classes:
                cprint("     - %s %s SAKNAS I EDUKONTO SHEET" % (current_klass, current_name), 'yellow')
                missing_language_klass.append(current_klass)
                missing_language_name.append(current_name)

        row = [current_klass, current_name, current_groups, current_pn, current_email, language]

        content.append(row)

    if len(missing_language_klass) > 0:
        missing_language_dict = {
            'Klass': missing_language_klass,
            'Namn': missing_language_name
        }
        missing_language_df = pd.DataFrame.from_dict(missing_language_dict)
        time_stamp = str(datetime.datetime.now())[:10] # 2019-09-04
        filename = time_stamp + '_språk_som_saknas.csv'
        try:
            missing_language_df.to_csv(filename, sep=";", index=False)
            print("     [Saved info about missing language to file ", end="")
            cprint("'%s']" % (filename), 'cyan')
        except PermissionError:
            print(
                f"     SAVING FILE '{filename}' PERMISSION ERROR: Is file already open?")
        except:
            print("     Something went wrong!")
            
            
    return content

service = authenticate()

ID ='1tnG_zT2AhQk9hTBQYMyrX2_TGSeryHwG1mh96vtdqM4'
sheet_name = 'elevlista'
col_map = {
    'Elev Klass': 'Klass',
    'Elev Namn': 'Namn',
    'Elev Grupper': 'Grupper',
    'Elev Personnummer': 'Personnummer',
    'Elev Mail': 'Mail',
}

df_elevlista, errors = get_sheet_as_df(service, ID, sheet_name+'!A1:E', col_map)

# print(df_elevlista);

ID ='1tnG_zT2AhQk9hTBQYMyrX2_TGSeryHwG1mh96vtdqM4'
sheet_name = 'sva_sv'
col_map = {
    'Elev': 'Namn',
    'Personnummer': 'Personnummer',
    'Språk': 'Språk'
}

df_language, errors = get_sheet_as_df(service, ID, sheet_name+'!A1:E', col_map)

content = check_language(service, df_elevlista, df_language, ID)
# print(df_language)
print(len(content))

Checking language:  28%|██▊       | 190/684 [00:00<00:00, 867.70it/s]

     - 1C Al Kadumi, Fatimah SAKNAS I EDUKONTO SHEET
     - 1D Alhussein, Mohamad SAKNAS I EDUKONTO SHEET
     - 1D Alkisie, Ali SAKNAS I EDUKONTO SHEET
     - 1D Alnaami, Nawal SAKNAS I EDUKONTO SHEET
     - 1D Andersson, Liv SAKNAS I EDUKONTO SHEET
     - 1D Berglund, Sigrid SAKNAS I EDUKONTO SHEET
     - 1D Bergström, Siri SAKNAS I EDUKONTO SHEET
     - 1D Braimaj, Levina SAKNAS I EDUKONTO SHEET
     - 1D Darweesh, Redha SAKNAS I EDUKONTO SHEET
     - 1D Elg Ås, Eline SAKNAS I EDUKONTO SHEET
     - 1D Eriksson, Elvira SAKNAS I EDUKONTO SHEET
     - 1D Haj Mousa, Narjes SAKNAS I EDUKONTO SHEET
     - 1D Hall, Fox SAKNAS I EDUKONTO SHEET
     - 1D Hansson, Elton SAKNAS I EDUKONTO SHEET
     - 1D Hedlund, Noah SAKNAS I EDUKONTO SHEET
     - 1D Khalaf, Siamet SAKNAS I EDUKONTO SHEET
     - 1D Olsson, Elvira SAKNAS I EDUKONTO SHEET
     - 1D Sandberg, Vega SAKNAS I EDUKONTO SHEET
     - 1D Sander, Troy SAKNAS I EDUKONTO SHEET
     - 1D Smedh, Joline SAKNAS I EDUKONTO SHEET
     - 1D Stoo

Checking language:  56%|█████▋    | 386/684 [00:00<00:00, 915.74it/s]

     - 4D Andersson, Oliver SAKNAS I EDUKONTO SHEET
     - 4D Braimaj, Jolina SAKNAS I EDUKONTO SHEET
     - 4D Braimaj, Leon SAKNAS I EDUKONTO SHEET
     - 4D Eriksson, Holly SAKNAS I EDUKONTO SHEET
     - 4D Eriksson, Wilmer SAKNAS I EDUKONTO SHEET
     - 4D Haikola, Alice SAKNAS I EDUKONTO SHEET
     - 4D Haj Mousa, Tamer SAKNAS I EDUKONTO SHEET
     - 4D Hermansson, Julia SAKNAS I EDUKONTO SHEET
     - 4D Johansson, Hugo SAKNAS I EDUKONTO SHEET
     - 4D Kjellberg, Julia SAKNAS I EDUKONTO SHEET
     - 4D Lund, Filippa SAKNAS I EDUKONTO SHEET
     - 4D Mohamed Hassan, Sumayo SAKNAS I EDUKONTO SHEET
     - 4D Nilsson, Alva SAKNAS I EDUKONTO SHEET
     - 4D Nilsson, Elin SAKNAS I EDUKONTO SHEET
     - 4D Schnaars, Katelin SAKNAS I EDUKONTO SHEET
     - 4D Tiurinen, Oscar SAKNAS I EDUKONTO SHEET
     - 5D Abrahamsson, Lovisa SAKNAS I EDUKONTO SHEET
     - 5D Ak, Eren Azad SAKNAS I EDUKONTO SHEET
     - 5D Alashtar, Odai SAKNAS I EDUKONTO SHEET
     - 5D Alhussein, Aya SAKNAS I EDUKONTO

Checking language: 100%|██████████| 684/684 [00:00<00:00, 976.25it/s]

     [Saved info about missing language to file '2020-11-25_språk_som_saknas.csv']
684
